In [ ]:
!pip install pytorch-lightning -quiet

In [1]:
BATCH_SIZE = 200


In [2]:
%load_ext autoreload
%autoreload 1

%aimport model

import torch
import torchvision
import torchvision.transforms as transforms

import pytorch_lightning as pl

from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

base_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

data_augmentation = transforms.Compose(
    [transforms.RandomResizedCrop((32, 32), scale=(0.9, 1.0), ratio=(0.9, 1.1)),
     transforms.RandomHorizontalFlip(),
     transforms.RandomRotation(5)]
)


trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=base_transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=base_transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2, pin_memory=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2, pin_memory=True)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Files already downloaded and verified
Files already downloaded and verified


###MLP

In [3]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    check_finite=True
)

logger = TensorBoardLogger('tb_logs', name='mlp', log_graph=True, default_hp_metric=True)

net = model.MLPCifar10(16 * 2, 4, 0.2, transforms=data_augmentation)



TypeError: __init__() got an unexpected keyword argument 'transforms'